In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import (r2_score, auc, roc_auc_score, balanced_accuracy_score, 
                             average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve)
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.utils.validation import check_memory
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.simplefilter('ignore')

/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/guq/miniconda3/envs/python3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
import os
import sys

In [3]:
import numpy as np
import pandas as pd
import re

import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# result

In [4]:
work_dir = '/Users/guq/Documents/ohsu/drug_respond/results/smmart_nestedCV_2019/'
sub1 = 'Hyperparameter_xgbc/'
sub2 = 'Hyperparameter_xgbr/'
sub3 = 'Hyperparameter_rfc/'
sub4 = 'Hyperparameter_iraps'
sub5 = 'Hyperparameter_ridge'
sub6 = 'Hyperparameter_knr'
sub7 = 'Hyperparameter_svr'

In [5]:
smmart_drugs = ('Afatinib', 'Bortezomib', 'Cabozantinib', 'Dasatinib', 'Erlotinib',
                'Idelalisib', 'Imatinib', 'Olaparib', 'Palbociclib', 'Ponatinib',
                'Rapamycin', 'Sorafenib', 'Sunitinib', 'Trametinib', 'Vorinostat')

# concate nestedcv results
def concate_best_result(folder, file_name, scorer, classifier, results, discretize_z_score=-1, exact_z_score=False):
    
    if not file_name.endswith('tabular'):
        return results
    path = os.path.join(folder, file_name)
    res = pd.read_csv(path, sep='\t')
    
    res_filtered = res
    if exact_z_score:
        #print(path)
        if classifier == 'IRAPSClassifier':
            res_filtered = res_filtered[res_filtered['param_estimator__discretize'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = 'IRAPSClassifier_05'
        else:
            res_filtered = res_filtered[res_filtered['param_estimator__z_score'] == discretize_z_score]
            if discretize_z_score != -1:
                classifier = classifier + '_05'
        #print(classifier)
    res_sort = res_filtered.sort_values(['mean_test_'+scorer, 'std_test_'+scorer], ascending=[False, True])
    #res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer,'params']].head(1).reset_index(drop=True)
    res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer]].head(1).reset_index(drop=True)
    if not res_best['mean_test_'+scorer].isnull().any():
        if file_name.startswith((smmart_drugs)):
            res_best.insert(loc=0, column='dataset', value=file_name[:-8])
        else:
            res_best.insert(loc=0, column='dataset', value=file_name[:-11])
        res_best.insert(loc=0, column='classifier', value=classifier)
        if results is None:
            results = res_best
        else:
            results = results.append(res_best, ignore_index=True)
    else:
        print("empty result: %s in %s for %s" % (scorer, file_name, classifier))
    return results

In [6]:
# best AP scores
files1 = os.listdir(work_dir+sub1)

files2 = os.listdir(work_dir+sub2)

files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
files5 = os.listdir(work_dir+sub5)
files6 = os.listdir(work_dir+sub6)
files7 = os.listdir(work_dir+sub7)
results = None
scorer = 'binarize_average_precision_scorer'
for fl in files1:
    results = concate_best_result(work_dir+sub1, fl, scorer, 'XGBClassifier', results)

for fl in files2:
    results = concate_best_result(work_dir+sub2, fl, scorer, 'XGBRegressor', results)

    
for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'RandomForestClassifier', results)

for fl in files4:
    results = concate_best_result(work_dir+sub4, fl, scorer, 'IRAPS', results)
    
for fl in files5:
    results = concate_best_result(work_dir+sub5, fl, scorer, 'Ridge', results)
    
for fl in files6:
    results = concate_best_result(work_dir+sub6, fl, scorer, 'KNeighborsRegressor', results)

for fl in files7:
    results = concate_best_result(work_dir+sub7, fl, scorer, 'SVR', results)


results = results.sort_values(['classifier', 'dataset'])
results

empty result: binarize_average_precision_scorer in Erlotinib_1.tabular for IRAPS
empty result: binarize_average_precision_scorer in Sorafenib_30.tabular for IRAPS
empty result: binarize_average_precision_scorer in Imatinib_34.tabular for IRAPS


,classifier,dataset,mean_test_binarize_average_precision_scorer,std_test_binarize_average_precision_scorer
58,IRAPS,Afatinib_1032,0.332378,0.058300
52,IRAPS,Afatinib_1377,0.323348,0.022001
53,IRAPS,Bortezomib_104,0.239005,0.049228
55,IRAPS,Cabozantinib_249,0.290785,0.078369
59,IRAPS,Dasatinib_51,0.304665,0.119612
56,IRAPS,Idelalisib_238,0.351488,0.038993
64,IRAPS,Olaparib_1017,0.249302,0.094653
57,IRAPS,Olaparib_1495,0.197007,0.040389
60,IRAPS,Palbociclib_1054,0.306144,0.061868
62,IRAPS,Ponatinib_155,0.403744,0.056366


In [7]:
results.to_csv('/Users/guq/Documents/ohsu/drug_respond/results/smmart_nestedCV_2019/ap.csv', sep=',', index=False)

In [8]:
# best roc-auc scores
files1 = os.listdir(work_dir+sub1)

files2 = os.listdir(work_dir+sub2)

files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
files5 = os.listdir(work_dir+sub5)
files6 = os.listdir(work_dir+sub6)
files7 = os.listdir(work_dir+sub7)
results_auc = None
scorer = 'binarize_auc_scorer'
for fl in files1:
    results_auc = concate_best_result(work_dir+sub1, fl, scorer, 'XGBClassifier', results_auc)
    
for fl in files2:
    results_auc = concate_best_result(work_dir+sub2, fl, scorer, 'XGBRegressor', results_auc)
    
for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'RandomForestClassifier', results_auc)

for fl in files4:
    results_auc = concate_best_result(work_dir+sub4, fl, scorer, 'IRAPS', results_auc)
    
for fl in files5:
    results_auc = concate_best_result(work_dir+sub5, fl, scorer, 'Ridge', results_auc)
    
for fl in files6:
    results_auc = concate_best_result(work_dir+sub6, fl, scorer, 'KNeighborsRegressor', results_auc)

for fl in files7:
    results_auc = concate_best_result(work_dir+sub7, fl, scorer, 'SVR', results_auc)

results_auc = results_auc.sort_values(['classifier', 'dataset'])
results_auc

empty result: binarize_auc_scorer in Erlotinib_1.tabular for IRAPS
empty result: binarize_auc_scorer in Sorafenib_30.tabular for IRAPS
empty result: binarize_auc_scorer in Imatinib_34.tabular for IRAPS


,classifier,dataset,mean_test_binarize_auc_scorer,std_test_binarize_auc_scorer
58,IRAPS,Afatinib_1032,0.839321,0.045345
52,IRAPS,Afatinib_1377,0.814168,0.036338
53,IRAPS,Bortezomib_104,0.608697,0.121799
55,IRAPS,Cabozantinib_249,0.710768,0.117422
59,IRAPS,Dasatinib_51,0.617570,0.144060
56,IRAPS,Idelalisib_238,0.709146,0.083887
64,IRAPS,Olaparib_1017,0.705509,0.044018
57,IRAPS,Olaparib_1495,0.650515,0.063507
60,IRAPS,Palbociclib_1054,0.657479,0.036322
62,IRAPS,Ponatinib_155,0.869493,0.043455


In [9]:
data1 = go.Bar(
    x = results[results['classifier'] == 'XGBClassifier']['dataset'],
    y = results[results['classifier'] == 'XGBClassifier']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBClassifier']['std_test_binarize_average_precision_scorer']),  
    name = 'xgbc_AP'
)

data2 = go.Bar(
    x = results[results['classifier'] == 'XGBRegressor']['dataset'],
    y = results[results['classifier'] == 'XGBRegressor']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'XGBRegressor']['std_test_binarize_average_precision_scorer']),
    name = 'xgbr_AP'
)

data3 = go.Bar(
    x = results[results['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results[results['classifier'] == 'RandomForestClassifier']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'RandomForestClassifier']['std_test_binarize_average_precision_scorer']),
    name = 'rfc_AP'
)

data4 = go.Bar(
    x = results[results['classifier'] == 'IRAPS']['dataset'],
    y = results[results['classifier'] == 'IRAPS']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'IRAPS']['std_test_binarize_average_precision_scorer']),
    name = 'iraps_AP'
)


data5 = go.Bar(
    x = results[results['classifier'] == 'Ridge']['dataset'],
    y = results[results['classifier'] == 'Ridge']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'Ridge']['std_test_binarize_average_precision_scorer']),
    name = 'ridge_AP'
)

data6 = go.Bar(
    x = results[results['classifier'] == 'KNeighborsRegressor']['dataset'],
    y = results[results['classifier'] == 'KNeighborsRegressor']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'KNeighborsRegressor']['std_test_binarize_average_precision_scorer']),
    name = 'knr_AP'
)

data7 = go.Bar(
    x = results[results['classifier'] == 'SVR']['dataset'],
    y = results[results['classifier'] == 'SVR']['mean_test_binarize_average_precision_scorer'],
    error_y = dict(
        type='data',
        array=results[results['classifier'] == 'SVR']['std_test_binarize_average_precision_scorer']),
    name = 'svr_AP'
)

data11 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBClassifier']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBClassifier']['std_test_binarize_auc_scorer']),
    name = 'xgbc_ROC'
)


data12 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBRegressor']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBRegressor']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'XGBRegressor']['std_test_binarize_auc_scorer']),
    name = 'xgbr_ROC'
)


data13 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'RandomForestClassifier']['std_test_binarize_auc_scorer']),
    name = 'rfc_ROC'
)

data14 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'IRAPS']['dataset'],
    y = results_auc[results_auc['classifier'] == 'IRAPS']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'IRAPS']['std_test_binarize_auc_scorer']),
    name = 'iraps_ROC'
)

data15 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'Ridge']['dataset'],
    y = results_auc[results_auc['classifier'] == 'Ridge']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'Ridge']['std_test_binarize_auc_scorer']),
    name = 'ridge_ROC'
)

data16 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'KNeighborsRegressor']['dataset'],
    y = results_auc[results_auc['classifier'] == 'KNeighborsRegressor']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'KNeighborsRegressor']['std_test_binarize_auc_scorer']),
    name = 'knr_ROC'
)

data17 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'SVR']['dataset'],
    y = results_auc[results_auc['classifier'] == 'SVR']['mean_test_binarize_auc_scorer'],
    error_y = dict(
        type='data',
        array=results_auc[results_auc['classifier'] == 'SVR']['std_test_binarize_auc_scorer']),
    name = 'svr_ROC'
)



layout = go.Layout(
    xaxis=dict(
        title='Dataset'
    ),
    yaxis=dict(
        title='Performance score'
    ),
    barmode = 'group'
)
fig = go.Figure(data=[data1, data2,  data3, data4, data5, data6, data7], layout=layout)
iplot(fig)
#iplot(fig, filename='/Users/guq/Downloads/newplot_ap_1.png', image='png')


fig = go.Figure(data=[data11, data12, data13, data14, data15, data16, data17], layout=layout)
#iplot(fig, filename='/Users/guq/Downloads/newplot_roc_1.png', image='png')
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [10]:
trace1 = {
    "type": 'violin',
    "x": results['classifier'],
    "y": results['mean_test_binarize_average_precision_scorer'],
    "legendgroup": 'AP',
    "scalegroup": 'AP',
    "name": 'AP',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'blue'
    }
}

trace2 = {
    "type": 'violin',
    "x": results_auc['classifier'],
    "y": results_auc['mean_test_binarize_auc_scorer'],
    "legendgroup": 'ROC-AUC',
    "scalegroup": 'ROC-AUC',
    "name": 'ROC-AUC',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'pink'
    }
}

layout = {
    "yaxis": {
        "zeroline": False,
    },
    "violinmode": 'group'
}
fig = go.Figure(data=[trace1, trace2], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/